# GEOS-CF in Google Earth Engine


This notebook details methods for interacting with GEOS Composition Forecast (GEOS-CF) model diagnostics via the Google Earth Engine (GEE) Python API.

This API allows users to access a huge repository of data from GEE and use GEE functions to run fast server-side functions to manipulate data.

The strength of the GEE Python API is that users can pull data client-side and interact with selected fields within Pandas dataframes and other useful data structures.

This notebook will go through the following steps:

- Access GEOS-CF historical estimates and TROPOMI observations from the GEE data repository
- Turning an image collection into a Pandas dataframe at a selected point-of-interest
- Merging GEOS-CF model estimates of NO2 with TROPOMI observations of NO2
- Visualizing these data as images on an interactive folium map
- Visualizing these data using interactive plotly plots
- Gap-filling TROPOMI observations of tropospheric NO2 data with an XGBoost model


Notebook created by Callum Wayman, 2023

## Importing Required Modules

The Python Earth Engine API requires installation, authentican, and initialization in order to run in a Python environment. These steps can be accomplished in command line, or within Python code. 

More information on these steps can be found [here.](https://developers.google.com/earth-engine/guides/python_install)

Other modules required for this tutorial are listed below.

In [1]:
import ee

#ee.Authenticate()#force=True)

ee.Initialize(project='ee-callumwayman-cf')

Enter verification code: 4/1AQlEd8xgLUCLoAO_fZeHySCF-L8nXQ1AurcfgR4J_yNiBQVlPef727-0dqs

Successfully saved authorization token.


In [3]:
import datetime as dt
from math import sqrt
import requests

import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

import folium
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import plotly.express as px
import shap
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'shap'

## Importing Data from Google Earth Engine

Two collections are imported in this tutorial. The GEOS-CF time-averaged hourly replay collection is being imported, and more information on this collection can be found [here.](https://developers.google.com/earth-engine/datasets/catalog/NASA_GEOS-CF_v1_rpl_tavg1hr)

The other collection being imported is the Near Real-Time Nitrogen Dioxide image collection from the TROPOMI instrument. This collection offers observations of various atmospheric conditions and parameters, including NO2, which will be the focus of this tutorial.

In [ ]:
# Import Data

# GEE Data
geosCf = ee.ImageCollection("NASA/GEOS-CF/v1/rpl/tavg1hr")
tropomi = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2")

## Turning Earth Engine Objects into Pandas Dataframes

Google Earth Engine data is often presented in two object types: features and images.

Features and images may be stacked into feature collections and image collections.

These object types are extremely useful within GEE to manipulate large data sets quickly, however they offer some limitations in our ability to manipulate the data within when we use Python.

For this reason, this tutorial will geographically subset the imported image collections and transform them into Pandas dataframes. More information on Pandas dataframes can be found [here.](https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/frame.html)

In [ ]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')
        
    # Convert the time field into a datetime.
    # Time values are stored differently for TROPOMI and GEOS-CF data
    # Need to remove the time columns from GEOS-CF dataframes
    if 'number' in list_of_bands[0]:
        df['datetime'] = pd.to_datetime(df['time'], unit='ms').dt.strftime('%Y-%m-%d %H:%M')
    else:
        df['datetime'] = pd.to_datetime(df['time'], unit='ms')
        # Remove time columns from GEOS-CF dataframes
        df.pop('time')

    # Keep the columns of interest.
    df = df[['datetime',  *list_of_bands]]

    df.reset_index(inplace=True)
    df.drop(columns='index', inplace=True)
    
    return df

In [ ]:
class ee_subset:
    """
    A class with methods to create subsets of Earth Engine
    image collections.
      
    ...

    Attributes
    ----------
    band_dict : dict
        dictionary whose keys are band names and values are unit
        conversion values
    num_days : int
        number of days to be subset

    Methods
    -------
    get_subset_collection
        Returns a GEE collection subset for the number of days
        specified.
    get_point_df
        Takes a GEE image collection and returns a dataframe subset for the 
        number of days at a specific set of coordinates.
    """
    def __init__(self, band_dict, num_days):
        # Save band names and unit conversion values
        self.band_dict = band_dict
        self.band_list = list(band_dict.keys())
        
        # Get Dates
        # All subsets end at June 15th, 2023 for this tutorial
        f_date_datetime = dt.datetime.strptime('2023-06-15', "%Y-%m-%d")
        f_year = f_date_datetime.year
        f_month = f_date_datetime.month
        f_day = f_date_datetime.day
        i_date_datetime = f_date_datetime - dt.timedelta(num_days)
        i_year = i_date_datetime.year
        i_month = i_date_datetime.month
        i_day = i_date_datetime.day

        # Initial date of interest (inclusive).
        self.i_date = f'{i_year}-{i_month}-{i_day}'

        # Final date of interest (exclusive).
        self.f_date = f'{f_year}-{f_month}-{f_day}'
    
    def get_subset_collection(self, collection):
        """
        Subsets a GEE collection and returns subset 
        with appropriate time-window and bands selected.
        
        Parameters
        ----------
        collection : ee.imagecollection.ImageCollection
            GEE image collection to be subset
        

        Returns
        -------
        coll_subset : ee.imagecollection.ImageCollection
            Subset of collection
        """

        # Select bands and dates for collection
        coll_subset = collection.select(self.band_list).filterDate(self.i_date, self.f_date)

        return coll_subset
        
    
    def get_point_df(self, collection, lat, lon):
        """
        Subsets a GEE collection and returns subset 
        dataframe for a specific set of coordinates.
        
        Parameters
        ----------
        collection : ee.imagecollection.ImageCollection
            GEE image collection to be subset
        lat : int
            Latitude coordinate for point of interest
        lon : int
            Longitude coordinate for point of interest
        

        Returns
        -------
        data_features: pandas.core.frame.DataFrame
            Subset of collection returned as data frame
            for selected bands, time window, and point of interest.
        """
        
        # Get subsetted collection
        coll_subset = self.get_subset_collection(collection)
        
        # EE point from lat, lon
        poi = ee.Geometry.Point(lon, lat)
        
        # Scale in meters
        scale = 1000
        
        # Get the data for the pixel intersecting the point
        data_poi = coll_subset.getRegion(poi, scale).getInfo()
        
        # Call function to turn data into dataframe
        data_features = ee_array_to_df(data_poi, self.band_list)
        
        # Convert feature units to desired units for each band
        for b in self.band_list:
            data_features[b] = data_features[b]*self.band_dict[b]

        return data_features

## Subsetting GEE Data

Select the desired bands you wish to analyze from each collection. In this tutorial, we will be working with tropospheric column and surface level NO2 data.

We select relevant CF and TROPOMI bands, lat/lon, and number of days and subset the data using the ee_subset class and associated methods.

In [ ]:
# Set selected bands
cfSurfBand = 'NO2' # mol mol-1
cfTropBand = 'TROPCOL_NO2' # 1.0e15 molec cm-2
tropNo2Band = 'tropospheric_NO2_column_number_density' # mol/m2

# Create band dictionaries to store unit conversion information
cf_chm_band_dict = {cfSurfBand: 1.0e9, cfTropBand: 10000*1e15/6.02e23, 'O3': 1.0e9, 'NOy': 1.0e9, 'PM25_RH35_GCC': 1}
cf_met_band_dict = {'T10M': 1, 'ZPBL': 1, 'U10M': 1, 'V10M': 1, 'RH': 1}
trop_band_dict = {tropNo2Band: 1}

# Change to exact lat/lon for takoma rec
lat = 38.97
lon = -77.02

# EE point from lat, lon
poi = ee.Geometry.Point(lon, lat)

#Number of days to visualize
num_days = 300

# Get subset GEE collection and subset Dataframe

# GEOS-CF Chemistry
cf_chm_subset = ee_subset(cf_chm_band_dict, num_days)
cf_chm_subset_collection = cf_chm_subset.get_subset_collection(geosCf)
cf_chm_features = cf_chm_subset.get_point_df(geosCf, lat, lon)

# GEOOS-CF Meteorology
cf_met_subset = ee_subset(cf_met_band_dict, num_days)
cf_met_subset_collection = cf_met_subset.get_subset_collection(geosCf)
cf_met_features = cf_met_subset.get_point_df(geosCf, lat, lon)

# TROPOMI chemistry
trop_subset = ee_subset(trop_band_dict, num_days)
trop_subset_collection = trop_subset.get_subset_collection(tropomi)
trop_features = trop_subset.get_point_df(tropomi, lat, lon)

## Mapping GEOS-CF and TROPOMI

Using the folium package, we can create leaflet maps that allow us to view the data we have accessed from the GEE data repository.

This section will also explore the use of plotly plots to show a time-series of the subsetted data from GEOS-CF and TROPOMI at the point of interest.

In [ ]:
# Define a point of interest (POI) with a buffer zone of 1000 km around POI.
roi = poi.buffer(1e6)

# Reduce the LST collection by mean.
cf_img = cf_chm_subset_collection.mean()

# Adjust for scale factor.
cf_img = cf_img.select(cfSurfBand).multiply(cf_chm_band_dict[cfSurfBand])

my_map = folium.Map(location=[lat, lon], zoom_start=10)

### Adding a Useful Folium Method

The below function creates a new folium method and adds it to the folium.Map class.

This function receives an Earth Engine image object and creates a set of folium tiles from that image which can be added to the map.

The new method allows us to easily visualize images on a basemap as we might in the GEE code editor.

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """
    Adds a method for displaying Earth Engine image tiles to folium map.
    
    Parameters
        ----------
        ee_image_object : ee.image.Image
            GEE image collection to be subset
        vis_params : dict
            Dictionary of GEE visualization parameters
        
        Returns
        -------
        None
    """
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
    
# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

### Creating an Interactive Time-Series Plot

In this section, we create and save a plotly plot showing a time-series of tropospheric NO2 data from GEOS-CF and TROPOMI.

The interactive plot will be added to a below folium map.

In [ ]:
# Make the figure object
fig=make_subplots(specs=[[{"secondary_y":True}]])

# Create the first plotly trace showing GEOS-CF tropospheric NO2
fig.add_trace(
    go.Scatter(
    x=cf_chm_features['datetime'],
    y=cf_chm_features[cfTropBand],
    name="Tropospheric NO2 (mol/m^2)",
    mode='lines+markers',
    hoverinfo='y',
    line = dict(color='indigo', width=3)
     ),
    secondary_y=False)

# Create the second plotly trace showing TROPOMI tropospheric NO2
fig.add_trace(
    go.Scatter(
    x=trop_features['datetime'],
    y=trop_features[tropNo2Band],
    name="TROPOMI NO2 (mol/m^2)",
    mode='markers',
    hoverinfo='y',
    line = dict(color='red', width=3)
     ),
    secondary_y=False)

# Update figure axis aesthetics and labels
fig.update_layout(hoverlabel_bgcolor='#DAEEED',  #Change the background color of the tooltip to light gray
             title_text="GEOS-CF Tropospheric NO2 Replay", #Add a chart title
             title_font_family="Times New Roman",
             title_font_size = 20,
             title_font_color="darkblue", #Specify font color of the title
             title_x=0.5, #Specify the title position
             xaxis=dict(
                    tickfont_size=10,
                    tickangle = 270,
                    showgrid = True,
                    zeroline = True,
                    showline = True,
                    showticklabels = True,
                    #dtick=86400000,
                    dtick="M1",
                    tickformat="%m/%d\n%Y"
                    ),
             legend = dict(orientation = 'h', xanchor = "center", x = 0.72, y= 1), #Adjust legend position
             yaxis_title='Tropospheric NO2 (mol/m^2)')

# Set scientific notation format for y-axis
fig.update_yaxes(exponentformat='e')

# Write the image to a local html file
fig.write_html('surface_no2.html')

In [ ]:
# Select tropospheric NO2 and date range for GEOS-CF and scale to ppbv
cf_trop_img = cf_chm_subset_collection.select(cfTropBand).filterDate('2023-05-14', '2023-05-15').mean().multiply(cf_chm_band_dict[cfTropBand])

# Select tropospheric NO2 and date range for TROPOMI
trop_img = trop_subset_collection.filterDate('2023-05-14', '2023-05-15').mean()

# Set visualization parameters for NO2
cf_vis_params = {
    'min': 1,'max': 40,
    'palette': ['white', 'purple'],
    'opacity': 0.5
}

# Visualization parameters for tropospheric column NO2
visTrop = {'min': 1e-6,
    'max': 1e-4,
    'palette': ['white', 'purple'],
    'opacity': 0.5
          }

# Create a map.
lat, lon = lat, lon
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add the cf and tropomi data to the map object.
my_map.add_ee_layer(cf_trop_img, visTrop, 'GEOS-CF NO2')
my_map.add_ee_layer(trop_img, visTrop, 'TROPOMI NO2')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Set the plotly-generated html file into a html snippet
html="""
    <iframe src=\"""" + 'surface_no2.html' + """\" width="850" height="400"  frameborder="0">    
    """
    
# Create pop-up with added plotly html snippet
popup = folium.Popup(folium.Html(html, script=True))

# Add marker to map for point of interest
marker = folium.Marker([lat, lon],
                       popup=popup).add_to(my_map)

# Display the map.
display(my_map)

## Notes

Consider a 3-hour CF window around tropomi overpass time instead of 1 to 1 correspondance.

Go back to old forecasts, how well did forecasts predict what tropomi would look like? 1 day out? 5 days out?

overfitting check can be the mean bias in training vs testing

target tropospheric column and then derive to surface and compare to openaq - done

use xgboost model to predict difference between model and obs

## XGBoost Model

In this section, we will use an XGBoost model to gap-fill TROPOMI observations of tropospheric column NO2.

This gap-filled data set can then be use to create an estimate of surface level NO2 values derived from observations.

In [ ]:
# merge chemistry and met features
features = cf_chm_features.merge(cf_met_features,on='datetime')

In [ ]:
features

In [ ]:
samp = features.resample('3H', on='datetime').mean()
samp.reset_index(inplace=True)

In [ ]:
# target variable is observation in ppbv
target = trop_features.copy()
target['datetime'] = pd.to_datetime(target['datetime'])
target["datetime"] = target["datetime"].dt.round("H")
#target.pop('time')
target

In [ ]:
#features=samp

#print(features, target)

# also add target observation
#merged = features.merge(target,on='datetime', how='left')
merged = features.merge(target, on='datetime', how='right')
#merged = pd.merge_asof(features, target, on='datetime', direction='nearest')
merged.fillna(value=np.nan, inplace=True)
#merged.resample('3H', on='datetime').mean()
#merged.reset_index(inplace=True)
#merged.pop('index')
merged

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
merged.plot(x = 'datetime', y = cfTropBand, ax = ax, label = 'GEOS-CF Tropospheric NO2') 
merged.plot(x = 'datetime', y = tropNo2Band, ax = ax, c = 'r', label = 'TROPOMI Tropospheric NO2', zorder=2)
ax.set_xlim([merged.datetime.min(), merged.datetime.max()])
ax.legend()
plt.show()

- need to create a data set of random points across the CF grid and tropomi grid

- each row has lat/lon as input

In [ ]:
# create X and Y for XGBoost
chm_vars = list(cf_chm_band_dict.keys())
met_vars = list(cf_met_band_dict.keys())
feature_names = chm_vars+met_vars
X = merged[feature_names]
Y = merged[[tropNo2Band]] * 10000

X_arr = X.to_numpy()
Y_arr = Y.to_numpy()[:,0]

print(X_arr.shape, Y_arr.shape)

X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)

dtrain = xgb.DMatrix(X_train, label=y_train, missing=np.nan, feature_names=feature_names)
dtest = xgb.DMatrix(X_test, label=y_test, missing=np.nan, feature_names=feature_names)

print(X, Y)

In [ ]:
# train model
print('BEGIN TRAINING')
train = xgb.DMatrix(X,Y)
params = {'booster' : 'gbtree', 'eta': 0.3, 'verbosity': 2}
model = xgb.train(params,dtrain)
train_pred = model.predict(dtrain)
prediction = model.predict(dtest)

In [ ]:
# statistics (on the trained data - this is cheating!)
yflat = np.array(y_train).ravel()
print('r2 = {:.2f}'.format(r2_score(yflat,train_pred)))
print('nrmse = {:.2f}'.format( sqrt(mean_squared_error(yflat,train_pred))/np.std(yflat)))
print('nmb = {:.2f}'.format(np.sum(train_pred-yflat)/np.sum(yflat)))

# plot
dat = pd.DataFrame({'obs':yflat,'pred':train_pred})
dat['date'] = merged['datetime']
dat['orig'] = merged[cfTropBand]

fig, ax = plt.subplots(figsize = (15, 10))
ax.plot(dat['date'],dat['obs'],color='black',label='Observed')
#ax.plot(dat['date'],dat['orig'],color='blue',label='GEOS-CF')
ax.plot(dat['date'],dat['pred'],color='red',label='GEOS-CF + XGBoost')
ax.legend()

plt.xticks(rotation=30)

In [ ]:
yflat = np.array(y_test).ravel()
print('r2 = {:.2f}'.format(r2_score(yflat,prediction)))
print('nrmse = {:.2f}'.format( sqrt(mean_squared_error(yflat,prediction))/np.std(yflat)))
print('nmb = {:.2f}'.format(np.sum(prediction-yflat)/np.sum(yflat)))

# plot
dat = pd.DataFrame({'obs':yflat,'pred':prediction})
dat['date'] = merged['datetime']
dat['orig'] = merged[cfTropBand]

fig, ax = plt.subplots(figsize = (15, 10))
ax.plot(dat['date'],dat['obs'],color='black',label='Observed')
#ax.plot(dat['date'],dat['orig'],color='blue',label='GEOS-CF')
ax.plot(dat['date'],dat['pred'],color='red',label='GEOS-CF + XGBoost')
ax.legend()

plt.xticks(rotation=30)

### Tuning the XGBoost Model Hyperparameters

In [ ]:
# https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

from sklearn.metrics import mean_absolute_error

# Compute MAE
mae_baseline = mean_absolute_error(y_test, prediction)
mae_baseline

In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta': 0.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

params['eval_metric'] = "mae"

# Set a high number of boost rounds
# We hope to find optimal number of rounds before this
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

In [ ]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

In [ ]:
cv_results

In [ ]:
cv_results['test-mae-mean'].min()

In [ ]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]
gridsearch_params

In [ ]:
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10,
        verbose_eval=0
    );    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min();
    boost_rounds = cv_results['test-mae-mean'].argmin();
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
        print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
params['max_depth'] = 10
params['min_child_weight'] = 6

In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [ ]:
min_mae = float("Inf")
best_params = None# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)

In [ ]:
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
params['subsample'] = 1.0
params['colsample_bytree'] = 1.0

In [ ]:
%time
# This can take some time…
min_mae = float("Inf")
best_params = None 
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))    
    # We update our parameters
    params['eta'] = eta    
    # Run and time CV
    %time cv_results = xgb.cv(params,dtrain,num_boost_round=num_boost_round,seed=42,nfold=5,metrics=['mae'],early_stopping_rounds=10)
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta

In [ ]:
print("Best params: {}, MAE: {}".format(best_params, min_mae))

In [ ]:
params['eta'] = 0.3

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

In [ ]:
# re-train with new hyperparameters

model = xgb.train(params,dtrain)
train_pred = model.predict(dtrain)
prediction = model.predict(dtest)

In [ ]:
# statistics (on the trained data - this is cheating!)
yflat = np.array(y_train).ravel()
print('r2 = {:.2f}'.format(r2_score(yflat,train_pred)))
print('nrmse = {:.2f}'.format( sqrt(mean_squared_error(yflat,train_pred))/np.std(yflat)))
print('nmb = {:.2f}'.format(np.sum(train_pred-yflat)/np.sum(yflat)))

# plot
dat = pd.DataFrame({'obs':yflat/1000,'pred':train_pred/1000})
dat['date'] = merged['datetime']
dat['orig'] = merged[cfTropBand]

fig, ax = plt.subplots(figsize = (15, 10))
ax.plot(dat['date'],dat['obs'],color='black',label='Observed')
ax.plot(dat['date'],dat['pred'],color='red',label='GEOS-CF + XGBoost')
ax.set_ylabel('Tropospheric NO2 (mol/m^2)')
ax.set_xlabel('Date')
ax.legend()

plt.xticks(rotation=30)

In [ ]:
yflat = np.array(y_test).ravel()
print('r2 = {:.2f}'.format(r2_score(yflat,prediction)))
print('nrmse = {:.2f}'.format( sqrt(mean_squared_error(yflat,prediction))/np.std(yflat)))
print('nmb = {:.2f}'.format(np.sum(prediction-yflat)/np.sum(yflat)))

# plot
dat = pd.DataFrame({'obs':yflat,'pred':prediction})
dat['date'] = merged['datetime']
dat['orig'] = merged[cfTropBand]

fig, ax = plt.subplots(figsize = (15, 10))
ax.plot(dat['date'],dat['obs'],color='black',label='Observed')
ax.plot(dat['date'],dat['pred'],color='red',label='GEOS-CF + XGBoost')
ax.set_ylabel('Tropospheric NO2 (mol/m^2)')
ax.set_xlabel('Date')
ax.legend()

plt.xticks(rotation=30)

In [ ]:
xgb.plot_importance(model)
plt.show()

In [ ]:
# How to derive shapley values
# Need to load JS vis in the notebook
#import shap
shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
i = 100
shap.force_plot(explainer.expected_value, shap_values[i], features=X.loc[i], feature_names=X.columns)

In [ ]:
shap.summary_plot(shap_values, features=X, feature_names=X.columns)

## Explaining the plot above

Interpretation is that a high value of a feature in red contributes to either higher or lower tropomi derived sfc no2. For example, high NO2 from GEOS-CF usually corresponds to a higher predicted sfc NO2, however higher ZPBL corresponds to a lower sfc NO2.

The goal of SHAP is to explain the prediction of an instance x by computing the contribution of each feature to the prediction. [...] SHAP feature importance is an alternative to permutation feature importance. There is a big difference between both importance measures: Permutation feature importance is based on the decrease in model performance. SHAP is based on magnitude of feature attributions.
(https://datascience.stackexchange.com/questions/99650/difference-between-feature-effect-and-feature-importance)

In other words, feature importance explains contribution to goodness of fit, and SHAP value explains contribution to predicted value.

In [ ]:
# Apply predictor to full input dataset
data = features.copy()[chm_vars+met_vars]
dtest = xgb.DMatrix(data)
ypred = model.predict(dtest)

# Make a long merged dataset to produce a gap-filled prediction of tropomi derived sfc no2
merged_long = features.merge(target.drop_duplicates(subset=['datetime']),on='datetime', how='left')
merged_long.fillna(value=np.nan, inplace=True)
merged_long['gap_filled_tropomi'] = ypred / 10000
merged_long

### Gap-filling TROPOMI with the XGBoost Model

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
merged_long.plot(x = 'datetime', y = cfTropBand, ax = ax, label = 'Tropospheric Column NO2') 
merged_long.plot.scatter(x = 'datetime', y = tropNo2Band, ax = ax, c = 'r', label = 'TROPOMI Observations', zorder=2)
merged_long.plot(x = 'datetime', y = 'gap_filled_tropomi', ax = ax, c = 'k', label = 'prediction')
ax.set_xlim([pd.to_datetime('2023-03-01'), pd.to_datetime('2023-06-01')])
ax.legend()
plt.show()

In [ ]:
# Plotting Tropospheric NO2 from TROPOMI and Surface Observations from Open AQ

merged_long_obs.rename(columns={tropNo2Band: 'TROPOMI Observations', cfTropBand: 'CF Tropospheric NO2'}, inplace=True)
melty = pd.melt(merged_long_obs, id_vars='datetime', value_vars = ['TROPOMI Observations', 'CF Tropospheric NO2'], var_name='NO2 Value Source')

fig = px.scatter(data_frame=melty, 
                    x='datetime', 
                    y='value', 
                    color = 'NO2 Value Source', 
                    title = 'Tropospheric NO2 Timeseries at Takoma Park, MD')
fig.update_yaxes(title= 'Tropospheric NO2 (mol/m^2)', exponentformat='e')

fig.update_layout(
    title=dict(font={'size':20}),
    legend=dict(font={'size':16})
)

fig.update_yaxes(title_font={'size':18},
                 tickfont=dict(size=16))
fig.update_xaxes(title_font={'size':18},
                 tickfont=dict(size=16),
                range=['2023-01-01','2023-01-08'])
fig.show()

In [ ]:
# Plotting Tropospheric NO2 from TROPOMI and Surface Observations from Open AQ

merged_long_obs.rename(columns={tropNo2Band: 'TROPOMI Observations', cfTropBand: 'CF Tropospheric NO2', 'gap_filled_tropomi': 'Gap Filled TROPOMI'}, inplace=True)
melty = pd.melt(merged_long_obs, id_vars='datetime', value_vars = ['TROPOMI Observations', 'CF Tropospheric NO2', 'Gap Filled TROPOMI'], var_name='NO2 Value Source')

fig = px.scatter(data_frame=melty, 
                    x='datetime', 
                    y='value', 
                    color = 'NO2 Value Source', 
                    title = 'Tropospheric NO2 Timeseries at Takoma Park, MD')
fig.update_yaxes(title= 'Tropospheric NO2 (mol/m^2)', exponentformat='e')

fig.update_layout(
    title=dict(font={'size':20}),
    legend=dict(font={'size':16})
)

fig.update_yaxes(title_font={'size':18},
                 tickfont=dict(size=16))
fig.update_xaxes(title_font={'size':18},
                 tickfont=dict(size=16),
                range=['2023-01-01','2023-01-08'])

fig.show()

### Creating Surface Data from TROPOMI Observations

Using a ratio technique we can create a dataset of estimated surface NO2 from predicted TROPOMI observations.

We will then compare those values to OpenAQ data for the same site.

In [ ]:
def get_sfc(target, target_band):
    target['sfc'] = target[target_band] * target[cfSurfBand] / target[cfTropBand]
    
    return target

In [ ]:
merged_long = get_sfc(merged_long, 'gap_filled_tropomi')

In [ ]:
merged_long

In [ ]:
#url = "https://api.openaq.org/v2/measurements?location_id=1704&parameter=no2&date_from=2022-03-01T08:37:22-04:00&date_to=2023-03-30T08:37:22-04:00&limit=100000"
url = f"https://api.openaq.org/v2/measurements?location_id=1704&parameter=no2&date_from={cf_chm_subset.i_date}&date_to={cf_chm_subset.f_date}&limit=100000"
r = requests.get( url )
assert(r.status_code==200)
rs = r.json()['results']
obs = pd.DataFrame()
obs = pd.json_normalize(r.json()['results'])
obs['datetime'] = [dt.datetime.strptime(i,'%Y-%m-%dT%H:%M:%S+00:00') for i in obs['date.utc']]
obs['open_aq_value'] = obs['value'] * 1000
obs['unit'] = 'ppb'
obs.head()

In [ ]:
#greater than the start date and smaller than the end date
mask = (obs.datetime.max() >= merged_long.datetime) & (merged_long.datetime >= obs.datetime.min())

obs_df = obs[['datetime', 'open_aq_value']]

merged_long_obs = merged_long.loc[mask].merge(obs_df,on='datetime', how='left')
merged_long_obs.fillna(value=np.nan, inplace=True)

merged_long_obs

In [ ]:
# Plotting Surface Derived NO2 from TROPOMI and Surface Observations from Open AQ

merged_long_obs.rename(columns={'sfc': 'TROPOMI Derived Obs', 'open_aq_value': 'Open AQ'}, inplace=True)
melty = pd.melt(merged_long_obs, id_vars='datetime', value_vars = ['TROPOMI Derived Obs', 'Open AQ'], var_name='NO2 Value Source')

fig = px.scatter(data_frame=melty, 
                    x='datetime', 
                    y='value', 
                    color = 'NO2 Value Source', 
                    title = 'Surface NO2 Timeseries at Takoma Park, MD')
fig.update_yaxes(title= 'Surface NO2 (ppb)')

fig.show()

In [ ]:
# Calcuate RMSE for Two Data Sets

diff = merged_long_obs['TROPOMI Derived Obs'] - merged_long_obs['Open AQ']
rmse = np.square(diff).sum()/len(diff)

print(rmse)

### Creating GEOS-CF Forecasts from the XGBoost Model

In [ ]:
import xarray as xr

ds = xr.open_dataset('https://tropomi.gesdisc.eosdis.nasa.gov/opendap/S5P_TROPOMI_Level2/S5P_L2__NO2____HiR.2/2023/276/S5P_OFFL_L2__NO2____20231003T005315_20231003T023444_30938_03_020500_20231004T164431.nc')

In [ ]:
import xarray as xr

url = 'http://opendap.nccs.nasa.gov:80/dods/gmao/geos-cf/assim/chm_tavg_1hr_g1440x721_v36'
url = 'http://opendap.nccs.nasa.gov:80/dods/gmao/geos-cf/assim/chm_tavg_1hr_g1440x721_v36'
#url = 'https://opendap.nccs.nasa.gov/dods/gmao/geos-cf/assim/chm_tavg_1hr_g1440x721_v36'
ds = xr.open_dataset(url, decode_times=False)

In [ ]:
ds